### Import

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import optuna
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler

### Data Load

In [5]:
import sys
import os
import pandas as pd

# 현재 작업 디렉토리 경로를 가져와 shared codes 폴더의 위치를 sys.path에 추가합니다.
# sys.path에 추가된 경로에 있는 py 폴더는 임포트할 수 있다.
current_dir = os.getcwd()
shared_codes_dir = os.path.join(current_dir, '../shared codes')
sys.path.append(shared_codes_dir)


# cover_nan 모듈을 임포트
from cover_nan_0215_dahun import missing_value_removal_function

# 원본 train 데이터 로드
train = pd.read_csv("../shared codes/data/train.csv")
test = pd.read_csv("../shared codes/data/test.csv")

# missing_value_removal_function 사용
train_young, train_middle, train_old, train_unknown = missing_value_removal_function(train)
test_young, test_middle, test_old, test_unknown = missing_value_removal_function(test)

✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!
✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!


In [31]:
def data_preprocessing(train, test):
    index_train = train['idx']
    X = train.drop(['임신 성공 여부', 'idx'], axis=1)
    y = train['임신 성공 여부']

    # index_test = test['idx']
    index_test = test['idx'].copy()  # Ensure a copy is made to prevent any shared memory issues
    test = test.drop('idx', axis=1)

    #Data Pre-processing
    # Categorical(범주형) 칼럼 찾기
    categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()

    # 데이터 인코딩
    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    X_train_encoded = X.copy()
    X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

    X_test_encoded = test.copy()
    X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

    # 불필요한 칼럼 삭제
    columns_to_drop = [
            "남성 주 불임 원인",
            "남성 부 불임 원인",
            "불임 원인 - 정자 농도",
            "불임 원인 - 정자 면역학적 요인",
            "불임 원인 - 정자 운동성",
            "불임 원인 - 정자 형태",
            '배란 유도 유형'
    ]
    X_train_encoded = X_train_encoded.drop(columns = columns_to_drop)    
    X_test_encoded = X_test_encoded.drop(columns = columns_to_drop)
    

    # 데이터 정규화 (X_train_encoded & X_test_encoded)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_encoded)
    X_test_scaled = scaler.transform(X_test_encoded)  # 동일한 스케일 적용

    # DataFrame 변환 (Feature 이름 유지)
    feature_names = [f"Feature_{i}" for i in range(X_train_scaled.shape[1])]
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_names)

    # 상관 행렬 계산
    correlation_matrix_train = X_train_scaled_df.corr()

    # 다중 공선성이 높은 칼럼 찾기 (절대 상관 계수가 0.8 이상)
    threshold = 0.8
    high_corr_features = set()

    for i in range(len(feature_names)):
        for j in range(i + 1, len(feature_names)):
            if abs(correlation_matrix_train.iloc[i, j]) > threshold:
                high_corr_features.add(feature_names[j])  # 공선성이 높은 컬럼 추가

    # 다중 공선성이 높은 컬럼 제거
    X_train_encoded = X_train_scaled_df.drop(columns=high_corr_features, errors='ignore')
    X_test_encoded = X_test_scaled_df.drop(columns=high_corr_features, errors='ignore')


    X_train_encoded['idx'] = index_train.reset_index(drop=True)
    X_test_encoded['idx'] = index_test.reset_index(drop=True)


    return X_train_encoded, X_test_encoded, y


In [32]:
train_young, train_middle, train_old, train_unknown = missing_value_removal_function(train)
test_young, test_middle, test_old, test_unknown = missing_value_removal_function(test)

X_train_encoded_young, X_test_encoded_young, y_young = data_preprocessing(train_young, test_young)
X_train_encoded_middle, X_test_encoded_middle, y_middle = data_preprocessing(train_middle, test_middle)
X_train_encoded_old, X_test_encoded_old, y_old = data_preprocessing(train_old, test_old)

✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!
✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!


NameError: name 'X_train_scaled_df' is not defined

In [30]:
X_train_encoded_young

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_9,Feature_10,...,Feature_37,Feature_38,Feature_41,Feature_42,Feature_44,Feature_45,Feature_46,Feature_47,Feature_48,idx
0,-1.732036,1.517565,0.136191,-0.910711,0.572873,-0.680849,-0.15317,-0.086829,-0.072521,-0.511059,...,-0.316811,-0.074932,-0.217518,-0.413544,-0.135082,-0.081968,0.0,-0.04008,-0.251576,0
1,-1.732005,0.022890,0.136191,0.703631,0.572873,-0.680849,-0.15317,-0.086829,-0.072521,-0.511059,...,-0.192517,-0.074932,-0.217518,-0.413544,-0.135082,-0.081968,0.0,-0.04008,-0.887744,2
2,-1.731975,0.022890,0.136191,-0.910711,0.572873,-0.680849,-0.15317,-0.086829,-0.072521,-0.511059,...,-0.316811,-0.074932,-0.217518,-0.413544,-0.135082,-0.081968,0.0,-0.04008,-0.251576,4
3,-1.731944,-0.475335,0.136191,-0.910711,-1.745589,1.468755,-0.15317,-0.086829,-0.072521,-0.511059,...,-1.186865,-0.074932,-0.217518,-0.413544,-0.135082,-0.081968,0.0,-0.04008,-0.887744,6
4,-1.731914,1.019340,0.136191,0.703631,0.572873,-0.680849,-0.15317,-0.086829,-0.072521,1.956722,...,-0.938278,-0.074932,-0.217518,-0.413544,-0.135082,-0.081968,0.0,-0.04008,-0.251576,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113722,1.731914,0.521115,0.136191,-0.910711,0.572873,-0.680849,-0.15317,-0.086829,-0.072521,-0.511059,...,-0.565398,-0.074932,-0.217518,-0.413544,-0.135082,-0.081968,0.0,-0.04008,-0.251576,256340
113723,1.731944,1.019340,0.136191,-0.910711,0.572873,1.468755,-0.15317,-0.086829,-0.072521,-0.511059,...,0.056070,-0.074932,-0.217518,-0.413544,-0.135082,-0.081968,0.0,-0.04008,1.020760,256343
113724,1.731975,1.517565,0.136191,0.703631,0.572873,1.468755,-0.15317,-0.086829,-0.072521,-0.511059,...,0.304656,-0.074932,-0.217518,-0.413544,-0.135082,-0.081968,0.0,-0.04008,-0.887744,256344
113725,1.732005,-1.471785,0.136191,-0.910711,0.572873,-0.680849,-0.15317,-0.086829,-0.072521,-0.511059,...,0.677537,-0.074932,2.038879,-0.413544,-0.135082,-0.081968,0.0,-0.04008,1.020760,256345


In [7]:
def optimize_models_and_get_best_params(X_train_encoded, y):

    def optimize_xgboost(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 5)
        }
        model = XGBClassifier(**params, random_state=42, eval_metric="logloss")
        scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")
        return np.mean(scores)


    def optimize_lightgbm(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 10, 50),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0)
        }
        model = LGBMClassifier(**params, random_state=42)
        scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
        return np.mean(scores)

    def optimize_random_forest(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 300),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_samples_split": trial.suggest_int("min_samples_split", 3, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4)
        }
        model = RandomForestClassifier(**params, random_state=42)
        scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
        return np.mean(scores)

    # Optuna 스터디 실행
    print("Optimizing XGBoost...")
    xgb_study = optuna.create_study(direction="maximize")
    xgb_study.optimize(optimize_xgboost, n_trials=100)
    best_xgb_params = xgb_study.best_params

    print("Optimizing LightGBM...")
    lgbm_study = optuna.create_study(direction="maximize")
    lgbm_study.optimize(optimize_lightgbm, n_trials=100)
    best_lgbm_params = lgbm_study.best_params

    print("Optimizing RandomForest...")
    rf_study = optuna.create_study(direction="maximize")
    rf_study.optimize(optimize_random_forest, n_trials=10)
    best_rf_params = rf_study.best_params

    params = [best_xgb_params, best_lgbm_params, best_rf_params]
    
    return params


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

def optimize_models_and_get_best_params_v2(X_train_encoded, y):

    def optimize_logistic_regression(trial):
        params = {
            "C": trial.suggest_float("C", 0.01, 10.0, log=True),
            "solver": trial.suggest_categorical("solver", ["liblinear", "lbfgs"]),
            "max_iter": trial.suggest_int("max_iter", 100, 1000),
        }
        model = LogisticRegression(**params, random_state=42)
        scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")
        return np.mean(scores)

    def optimize_random_forest(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 300),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_samples_split": trial.suggest_int("min_samples_split", 3, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4)
        }
        model = RandomForestClassifier(**params, random_state=42)
        scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
        return np.mean(scores)

    def optimize_xgboost(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 5)
        }
        model = XGBClassifier(**params, random_state=42, eval_metric="logloss")
        scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")
        return np.mean(scores)

    # Optuna 스터디 실행
    print("Optimizing Logistic Regression...")
    logreg_study = optuna.create_study(direction="maximize")
    logreg_study.optimize(optimize_logistic_regression, n_trials=50)
    best_logreg_params = logreg_study.best_params

    print("Optimizing Random Forest...")
    rf_study = optuna.create_study(direction="maximize")
    rf_study.optimize(optimize_random_forest, n_trials=10)
    best_rf_params = rf_study.best_params

    print("Optimizing XGBoost...")
    xgb_study = optuna.create_study(direction="maximize")
    xgb_study.optimize(optimize_xgboost, n_trials=50)
    best_xgb_params = xgb_study.best_params

    params = [best_logreg_params, best_rf_params, best_xgb_params]
    
    return params

In [9]:
def train_ensemble_model(X_train_encoded, y, X_test_encoded, params):

    
    # best params
    best_xgb_params, best_lgbm_params, best_rf_params = params

    # 최적화된 모델 생성
    xgb_model = XGBClassifier(**best_xgb_params, random_state=42, use_label_encoder=False, eval_metric="logloss")
    lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
    rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

    # Soft Voting 앙상블
    ensemble_model = VotingClassifier(
        estimators=[
            ("xgb", xgb_model),
            ("lgbm", lgbm_model),
            ("rf", rf_model)
        ],
        voting="soft"
    )

    # 전체 데이터로 학습
    ensemble_model.fit(X_train_encoded, y)

    pred_proba = ensemble_model.predict_proba(X_test_encoded)[:, 1]

    submission = pd.DataFrame({
       'probability': pred_proba,
       'idx': X_test_encoded['idx']
   })

    return submission

In [10]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

def train_ensemble_model_v2(X_train_encoded, y, X_test_encoded, params):

    # best params
    best_logreg_params, best_rf_params, best_xgb_params = params

    # 최적화된 모델 생성
    logreg_model = LogisticRegression(**best_logreg_params, random_state=42)
    rf_model = RandomForestClassifier(**best_rf_params, random_state=42)
    xgb_model = XGBClassifier(**best_xgb_params, random_state=42, use_label_encoder=False, eval_metric="logloss")

    # Soft Voting 앙상블
    ensemble_model = VotingClassifier(
        estimators=[
            ("logreg", logreg_model),
            ("rf", rf_model),
            ("xgb", xgb_model)
        ],
        voting="soft"
    )

    # 전체 데이터로 학습
    ensemble_model.fit(X_train_encoded, y)

    # 예측 확률
    pred_proba = ensemble_model.predict_proba(X_test_encoded)[:, 1]

    # 제출용 데이터프레임 생성
    submission = pd.DataFrame({
       'probability': pred_proba,
       'idx': X_test_encoded['idx']
    })

    return submission

In [12]:
def this_func_is_all_you_need(train, test):
    # missing_value_removal_function 사용
    train_young, train_middle, train_old, train_unknown = missing_value_removal_function(train)
    test_young, test_middle, test_old, test_unknown = missing_value_removal_function(test)

    X_train_encoded_young, X_test_encoded_young, y_young = data_preprocessing(train_young, test_young)
    X_train_encoded_middle, X_test_encoded_middle, y_middle = data_preprocessing(train_middle, test_middle)
    X_train_encoded_old, X_test_encoded_old, y_old = data_preprocessing(train_old, test_old)

    params_young = optimize_models_and_get_best_params_v2(X_train_encoded_young, y_young)
    params_middle = optimize_models_and_get_best_params_v2(X_train_encoded_middle, y_middle)
    params_old = optimize_models_and_get_best_params_v2(X_train_encoded_old, y_old)

    submission_young = train_ensemble_model_v2(X_train_encoded_young, y_young, X_test_encoded_young, params_young)
    submission_middle = train_ensemble_model_v2(X_train_encoded_middle, y_middle, X_test_encoded_middle, params_middle)
    submission_old = train_ensemble_model_v2(X_train_encoded_old, y_old, X_test_encoded_old, params_old)

    # test_unknown의 데이터 개수만큼 0으로 채워진 submission_unknown 생성
    submission_unknown = pd.DataFrame({
        'probability': [0] * len(test_unknown),
        'idx': test_unknown['idx']
    })

    submission = pd.concat([submission_young, submission_middle, submission_old, submission_unknown])

    submission = submission.sort_values('idx').reset_index(drop=True)
    submission = submission.drop(columns=['idx'])

    submission['ID'] = [f"TEST_{i:05d}" for i in range(len(submission))]

    submission = submission[['ID', 'probability']]

    submission.to_csv("submission.csv", index=False)

    return 

In [ ]:
# 원본 train 데이터 로드
train = pd.read_csv("../shared codes/data/train.csv")
test = pd.read_csv("../shared codes/data/test.csv")

this_func_is_all_you_need(train, test)